# Esercizio 5

Dato un file in formato GTF (Gene Transfer Format) che annota un set di geni sulla stessa genomica di riferimento, e il file della genomica di riferimento (*genomic reference*) in formato FASTA, produrre in output:

- (a scelta) le sequenze dei trascritti full-length o le sequenze delle coding sequences (CDS) che si trovano annotati nel file GTF
- il set dei geni (identificati tramite il loro HUGO name) relativi alle sequenze prodotte al punto precedente

L'*header* del formato FASTA deve contenere per ogni sequenza prodotta in output (trascritto full-length o CDS):        
- lo HUGO name del gene di riferimento
- l’identificatore del trascritto di riferimento (sia per trascritto full-length che per CDS)
- il tipo di sequenza (cioé se è trascritto full-length o CDS)
- lo strand del gene rispetto alla *genomic reference*
    
e deve rispettare il formato dei seguenti esempi:
         
    >ARHGAP4; U52112.4-003; len=3235 type=transcript; strand=-
    >AVPR2; U52112.2-003; type=cds; strand=+
   
***

Parametri in input:

- nome del file in formato GTF
- nome del file della *genomic reference* in formato FASTA
- nome della *feature* GTF che compone la sequenza da ricostruire: `exon` per i trascritti full-length e `CDS` per le coding sequences

***

Requisiti:

- deve essere definita una funzione `format_fasta()` che prenda come argomenti un'intestazione FASTA e una sequenza, e restituisca la sequenza in formato FASTA separata in righe di 80 caratteri.

- deve essere definita una funzione `reverse_complement()` che prenda come argomento una sequenza nucleotidica e ne restituisca il reverse&complement.

- deve essere definita una funzione `compose_feature()` che prenda come argomenti una lista di *features* dello stesso tipo (in cui ogni *feature* è una tupla *(start, end)*), la genomica di riferimento, lo strand delle *features* rispetto alla genomica di riferimento (cioé lo strand del gene a cui le *features* appartengono), ed effettui la concatenazione delle sequenze delle *features*, effettuando il reverse&complement se lo strand del gene è `-` (strand minus).

**NOTA BENE**: gli attributi (coppie *nome-valore*) del nono campo del file GTF non devono essere pensati a ordine fisso all'interno del campo. Per estrarre quindi un attributo, non si può usare il metodo `split()`, ma si deve necessariamente usare un'espressione regolare.

***

Variabili di output:
- `sequence_fasta_list`: lista delle sequenze ricostruite (trascritti full-length o coding sequences) in formato FASTA
- `gene_set`: set dei geni coinvolti nelle sequenze ricostruite

***

## Soluzione

Importare il modulo `re` per usare le espressioni regolari.

In [1]:
import re

### Definizione della funzione `format_fasta()`

In [2]:
def format_fasta(header, sequence):
    return header + '\n' + '\n'.join(re.findall('\w{1,80}', sequence))

**NOTA BENE**: supporre che l'*header* in input alla funzione non abbia il simbolo *newline* `\n` alla fine ma che abbia il simbolo `>` all'inizio.

### Definizione della funzione `reverse_complement()`

**NOTA BENE**: fare in modo che la funzione restituisca sempre una versione della sequenza in maiuscolo.

**NOTA BENE**: fare in modo che la funzione restituisca sempre una versione della sequenza in maiuscolo.

### Definizione della funzione `compose_feature()`

**NOTA BENE**: la concatenazione deve sempre avvenire per coordinate crescenti delle *features*.

### Parametri in input

In [5]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'
feature_name = 'CDS'

### Lettura della genomica di riferimento e memorizzazione nella variabile `genomic_reference`

Estrazione delle righe del file della genomica di riferimento nella lista `reference_file_rows`

In [7]:
reference_file_rows

['>ENm006\n',
 '...\n',
 ...]

Concatenazione delle righe contenenti la sequenza nucleotidica (dopo avere eliminato il simbolo di *newline* finale) nella variabile `genomic_reference`

In [9]:
genomic_reference

'...'

### Lettura dei *record* del file GTF e memorizzazione nella lista `gtf_file_rows`

In [11]:
gtf_file_rows

['ENm006\tVEGA_Known\texon\t71783\t71788\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t70312\t70440\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t69989\t70210\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64935\t65036\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64566\t64673\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52

### Filtraggio dei *record* GTF che servono per ricostruire le sequenze del tipo scelto

Tenere nella lista `gtf_file_rows` solo i *record* GTF che corrispondono al tipo di *feature* che compone la sequenza che si è scelto di ricostruire, cioé per i quali il terzo campo è uguale al valore della variabile `feature_name` (cioé `exon` se si è scelto di ricostruire i trascritti full-length e `CDS` se si è scelto di ricostruire le coding sequences).

In [13]:
gtf_file_rows

['ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64385\t64459\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72761\t72963\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72521\t72683\t.\t-\t1\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72253\t72315\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71872\t71965\t.\t-\t0\ttranscript_id "U52112.4

### Costruzione del dizionario degli strand dei geni e del set dei geni annotati

A partire dalla lista `gtf_file_rows`, costruire:
- il dizionario `strand_dict` degli strand dei geni:
    - *chiave*: HUGO name del gene
    - *valore*: strand del gene rispetto alla genomica di riferimento
    
- il set `gene_set` dei geni annotati relativamente al tipo di sequenza da ricostruire

**NOTA BENE**: il valore dello strand (settimo campo del *record* GTF) è costante lungo tutti i *record* relativi allo stesso gene.

Inizializzazione del dizionario vuoto.

Attraversare la lista `gtf_file_rows` per il riempimento del dizionario.

In [16]:
strand_dict

{'ARHGAP4': '-', 'AVPR2': '+'}

Estrarre dal dizionario `strand_dict` il set `gene_set` dei geni annotati.

In [18]:
gene_set

{'ARHGAP4', 'AVPR2'}

### Ricostruzione delle sequenze del tipo scelto

Costruire
- il dizionario `id_dict`:
    - *chiave*: HUGO name del gene
    - *valore*: set degli identificatori dei trascritti per cui è annotata il tipo di sequenza che si è scelto di ricostruire
- il dizionario `composition_dict`:
    - *chiave*: identificatore del trascritto
    - *valore*: lista delle *features* GTF, come tuple *(exon_start, exon_end)*, che compongono la sequenza da ricostruire

Inizializzare i dizionari vuoti.

Attraversare la lista `gtf_file_rows` e riempimento dei due dizionari.

In [21]:
id_dict

{'ARHGAP4': {'U52112.4-001',
  'U52112.4-003',
  'U52112.4-005',
  'U52112.4-010',
  'U52112.4-011',
  'U52112.4-017',
  'U52112.4-019',
  'U52112.4-020',
  'U52112.4-024'},
 'AVPR2': {'U52112.2-001', 'U52112.2-003'}}

In [22]:
composition_dict

{'U52112.4-005': [(71783, 71788),
  (70312, 70440),
  (69989, 70210),
  (64935, 65036),
  (64566, 64673),
  (64385, 64459)],
 'U52112.4-019': [(72761, 72963),
  (72521, 72683),
  (72253, 72315),
  (71872, 71965)],
 'U52112.4-017': [(72761, 72963),
  (72521, 72683),
  (72253, 72315),
  (71865, 71965)],
 'U52112.4-010': [(63857, 63942),
  (62286, 62346),
  (61857, 61991),
  (61663, 61768),
  (61328, 61561),
  (61169, 61242),
  (60898, 61081)],
 'U52112.4-020': [(72521, 72560),
  (71783, 71965),
  (70724, 70843),
  (70312, 70440),
  (70097, 70210)],
 'U52112.4-003': [(77293, 77359),
  (72761, 72965),
  (72521, 72683),
  (72253, 72315),
  (71783, 71965),
  (70312, 70440),
  (69989, 70210),
  (64935, 65036),
  (64566, 64757),
  (64375, 64459),
  (64181, 64208),
  (63857, 63959),
  (62286, 62346),
  (62079, 62156),
  (61857, 61991),
  (61663, 61768),
  (61328, 61561),
  (61169, 61242),
  (60898, 61081),
  (60600, 60692),
  (60227, 60326),
  (58886, 59119)],
 'U52112.4-001': [(77293, 77359),


Ricostruire le sequenze in formato FASTA a partire dai dizionari `id_dict`, `composition_dict` e `strand_dict` e memorizzarle nella lista `sequence_fasta_list`.

L'*header* deve essere del tipo:

    >ARHGAP4; U52112.4-003; len=3235 type=transcript; strand=-
    
se si è scelto di ricostruire i trascritti full-length, e:

    >AVPR2; U52112.2-003; len=?; type=cds; strand=+
    
se si è scelto di ricostruire le coding sequences (CDS).

Creare prima `sequence_fasta_list` come lista delle tuple *(header, sequenza)* (con *header* e sequenza separati) da trasformare poi nella lista di stringhe in formato FASTA.

In [24]:
sequence_fasta_list

[('>ARHGAP4; U52112.4-005; len=642 type=cds; strand=-',
  'GAGAAGCGGCAGGCCAAGTTCATGGAGCACAAACTCAAGTGCACAAAGGCGCGCAACGAGTACCTGCTTAGCCTGGCTAGTGTCAACGCTGCTGTCAGTAACTACTACCTGCATGACGTCTTGGACCTCATGGACTGCTGTGACACAGGGTTCCACCTGGCCCTGGGGCAGGTGCTCCGGAGCTACACGGCCGCTGAGAGCCGCACCCAAGCCTCCCAAGTGCAGGGCCTGGGCAGCCTGGAAGAAGCTGTGGAGGCCCTGGATCCTCCAGGGGACAAAGCCAAGGTTCTCGAGGTGCATGCTACCGTCTTCTGTCCCCCGCTGCGCTTTGACTACCACCCCCATGATGGGGATGAGGTGGCTGAGATCTGCGTTGAAATGGAGCTGCGGGACGAGATTCTGCCCAGAGCCCAGAACATCCAGAGCCGCCTGGACCGACAGACCATTGAGACAGAGGAGACCAGCCCCTCCACCGAGTCCCTCAAGTCCACCAGCTCAGACCCAGGCAGCCGGCAGGCGGGCCGGAGGCGCGGCCAGCAGCAGGAGACCGAAACCTTCTACCTCACGAAGCTCCAGGAGTATCTGAGTGGACGGAGCATCCTCGCCAAGCTGCAGGCCAAGCACGAGAAGCTGCAGGAGGCC'),
 ('>ARHGAP4; U52112.4-011; len=2778 type=cds; strand=-',
  'ATGGCCGCTCACGGGAAGCTGCGGCGGGAGCGGGGGCTGCAGGCTGAGTATGAGACGCAAGTCAAAGAGATGCGCTGGCAGCTGAGCGAGCAGCTGCGCTGCCTGGAGCTGCAGGGCGAGCTGCGGCGGGAGTTGCTGCAGGAGCTGGCAGAGTTCATGCGGCGCCGCGCTGAGGTGGAGCTGGAATACTCCCGGGGCCTGGAAAAGCTGGCCGAGCGCTTCTCCAGCCGTGGAG

Trasformare la lista di tuple `feature_fasta_list` in una lista `sequence_fasta_list` di sequenze in formato FASTA.

In [26]:
sequence_fasta_list

['>ARHGAP4; U52112.4-005; len=642 type=cds; strand=-\nGAGAAGCGGCAGGCCAAGTTCATGGAGCACAAACTCAAGTGCACAAAGGCGCGCAACGAGTACCTGCTTAGCCTGGCTAG\nTGTCAACGCTGCTGTCAGTAACTACTACCTGCATGACGTCTTGGACCTCATGGACTGCTGTGACACAGGGTTCCACCTGG\nCCCTGGGGCAGGTGCTCCGGAGCTACACGGCCGCTGAGAGCCGCACCCAAGCCTCCCAAGTGCAGGGCCTGGGCAGCCTG\nGAAGAAGCTGTGGAGGCCCTGGATCCTCCAGGGGACAAAGCCAAGGTTCTCGAGGTGCATGCTACCGTCTTCTGTCCCCC\nGCTGCGCTTTGACTACCACCCCCATGATGGGGATGAGGTGGCTGAGATCTGCGTTGAAATGGAGCTGCGGGACGAGATTC\nTGCCCAGAGCCCAGAACATCCAGAGCCGCCTGGACCGACAGACCATTGAGACAGAGGAGACCAGCCCCTCCACCGAGTCC\nCTCAAGTCCACCAGCTCAGACCCAGGCAGCCGGCAGGCGGGCCGGAGGCGCGGCCAGCAGCAGGAGACCGAAACCTTCTA\nCCTCACGAAGCTCCAGGAGTATCTGAGTGGACGGAGCATCCTCGCCAAGCTGCAGGCCAAGCACGAGAAGCTGCAGGAGG\nCC',
 '>ARHGAP4; U52112.4-011; len=2778 type=cds; strand=-\nATGGCCGCTCACGGGAAGCTGCGGCGGGAGCGGGGGCTGCAGGCTGAGTATGAGACGCAAGTCAAAGAGATGCGCTGGCA\nGCTGAGCGAGCAGCTGCGCTGCCTGGAGCTGCAGGGCGAGCTGCGGCGGGAGTTGCTGCAGGAGCTGGCAGAGTTCATGC\nGGCGCCGCGCTGAGGTGGAGCTGGAATACTCCCGGGGCCTGGAAAAGCTGGCCGAGCGCTTCTCCA